In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import torch

df = pd.read_csv('/content/drive/MyDrive/train.csv')
df = pd.
df = df.dropna()
df = df.reset_index(drop=True)
df.head()


,Class Index,Description
0,3,"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Reuters - Private investment firm Carlyle Grou...
2,3,Reuters - Soaring crude prices plus worries\ab...
3,3,Reuters - Authorities have halted oil export\f...
4,3,"AFP - Tearaway world oil prices, toppling reco..."


In [3]:
import nltk
import unicodedata
import re

def normalize_unicode(s):
    return unicodedata.normalize('NFD', s)

def preprocess_text(text):
    text = normalize_unicode(text)
    text = re.sub(r"(.)(\1{2,})", r"\1", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", " ", text)
    text = text.strip().lower()
    return text

nltk.download('punkt')

df['Description'] = df['Description'].apply(preprocess_text)
df['Description'] = df['Description'].apply(nltk.word_tokenize)
df['Description'] = df['Description'].apply(lambda x: ['<sos>'] + x + ['<eos>'])

df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Class Index,Description
0,3,"[<sos>, reuters, short, sellers, ,, wall, stre..."
1,3,"[<sos>, reuters, private, investment, firm, ca..."
2,3,"[<sos>, reuters, soaring, crude, prices, plus,..."
3,3,"[<sos>, reuters, authorities, have, halted, oi..."
4,3,"[<sos>, afp, tearaway, world, oil, prices, ,, ..."


In [4]:
# make the first 7600 entries of description in df the training set
train_set = df['Description']
pre_train_set = df['Description'][:7600]
pre_train_set = list(pre_train_set)
pre_val_set = df['Description'][7600:9000]
pre_val_set = list(pre_val_set)

In [5]:
print(pre_train_set)

[['<sos>', 'reuters', 'short', 'sellers', ',', 'wall', 'street', 's', 'dwindling', 'band', 'of', 'ultra', 'cynics', ',', 'are', 'seeing', 'green', 'again', '.', '<eos>'], ['<sos>', 'reuters', 'private', 'investment', 'firm', 'carlyle', 'group', ',', 'which', 'has', 'a', 'reputation', 'for', 'making', 'well', 'timed', 'and', 'occasionally', 'controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'placed', 'its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '.', '<eos>'], ['<sos>', 'reuters', 'soaring', 'crude', 'prices', 'plus', 'worries', 'about', 'the', 'economy', 'and', 'the', 'outlook', 'for', 'earnings', 'are', 'expected', 'to', 'hang', 'over', 'the', 'stock', 'market', 'next', 'week', 'during', 'the', 'depth', 'of', 'the', 'summer', 'doldrums', '.', '<eos>'], ['<sos>', 'reuters', 'authorities', 'have', 'halted', 'oil', 'export', 'flows', 'from', 'the', 'main', 'pipeline', 'in', 'southern', 'iraq', 'after', 'intelligence', 'showed', 'a', 'reb

In [6]:
import torch

In [7]:
from torch.utils.data import Dataset

class ELMO_Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    @staticmethod
    def create_vocab(data):
        vocab = set()
        for sentence in data:
            for word in sentence:
                vocab.add(word)

        vocab.add('<pad>')
        vocab.add('<unk>')
        vocab.add('<sos>')
        vocab.add('<eos>')
        return vocab

In [8]:
word_to_ix = {word: idx for idx, word in enumerate(ELMO_Dataset.create_vocab(train_set))}
print(len(word_to_ix))

59352


In [9]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    batch = [torch.tensor([word_to_ix.get(word, word_to_ix['<unk>']) for word in sent]) for sent in batch]
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=word_to_ix['<pad>'])

    input_tensor = padded_batch[:, :-1]
    target_truth = padded_batch[:, 1:]

    return input_tensor, target_truth

In [10]:
pre_train_dataset = ELMO_Dataset(pre_train_set)

pre_val_dataset = ELMO_Dataset(pre_val_set)

batch_size = 32

from torch.utils.data import DataLoader

pre_train_loader = DataLoader(pre_train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, pin_memory=True, num_workers=4)
pre_val_loader = DataLoader(pre_val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, pin_memory=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [11]:
import gensim
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

In [3]:
import numpy as np

vocab_size = len(word_to_ix)
embedding_dim = glove_vectors.vector_size
embedding_matrix = torch.zeros(vocab_size, embedding_dim)

special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']

average_vector = np.mean(glove_vectors.vectors, axis=0)

for word, i in word_to_ix.items():
    if word not in special_tokens:
        try:
            embedding_matrix[i] = torch.tensor(glove_vectors[word])
        except KeyError:
            embedding_matrix[i] = torch.tensor(average_vector)

    elif word == '<sos>' or word == '<eos>':
        embedding_matrix[i] = torch.randn(embedding_dim)
    elif word == '<unk>':
        embedding_matrix[i] = torch.tensor(average_vector)
    else:
        embedding_matrix[i] = torch.zeros(embedding_dim)

print(embedding_matrix.shape)

NameError: name 'word_to_ix' is not defined

In [1]:
import torch
import torch.nn as nn

class ELMO(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, num_layers, dropout):
        super(ELMO, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.shape[1], hidden_dim, num_layers, dropout=dropout, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_dim*2, embedding_matrix.shape[0])

    def forward(self, input_tensor):
        embedded = self.embedding(input_tensor)
        output, _ = self.lstm(embedded)
        output = self.linear(output)
        return output

In [2]:
model = ELMO(embedding_matrix, 300, 2, 0.5)

NameError: name 'embedding_matrix' is not defined

In [15]:
from tqdm import tqdm

In [18]:
def run_epoch(model, data_loader, loss_fn, epoch, optimizer=None):
    if optimizer:
        model.train()
    else:
        model.eval()

    total_loss = 0

    p_bar = tqdm(data_loader)
    for (input_tensor, target_truth) in p_bar:

        input_tensor = input_tensor.cuda()
        target_truth = target_truth.cuda()

        target_pred = []

        for i in range(input_tensor.size(1)):
            output_tensor = model(input_tensor[:, i].unsqueeze(1))
            target_pred.append(output_tensor)

        target_pred = torch.cat(target_pred, dim=1)
        loss = loss_fn(target_pred.view(-1, target_pred.size(-1)), target_truth.view(-1))
        total_loss += loss.item()

        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        mean_loss = total_loss / len(data_loader)

        p_bar.set_description(f'{"Train" if optimizer else "Validation"} Loss: {mean_loss:.4f}, count: {epoch}')


    return mean_loss

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

model.cuda()

best_val_loss = float('inf')

all_val_loss = []
all_train_loss = []


for epoch in range(num_epochs):
    train_loss = run_epoch(model, pre_train_loader, loss_fn, epoch+1, optimizer)
    all_train_loss.append(train_loss)
    with torch.no_grad():
        val_loss = run_epoch(model, pre_val_loader, loss_fn, epoch+1)
        all_val_loss.append(val_loss)

    print('Epoch: {}, Train Loss: {:.4f}, Val Loss: {:.4f}'.format(epoch+1, train_loss, val_loss))
    if val_loss < best_val_loss:
        counter = 0
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_lstm_model.pth')
    else:
        counter += 1
        if counter == 3:
            break

  0%|          | 0/238 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Train Loss: 0.3845, count: 1:  10%|▉         | 23/238 [00:11<01:50,  1.95it/s]


KeyboardInterrupt: ignored

In [ ]:
# pack_padded_sequence